In [68]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['Womens Clothing E-Commerce Reviews.csv']


In [69]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, Bidirectional
import matplotlib.pyplot as plt
from keras.layers import LSTM
from sklearn import metrics

from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [70]:
df = pd.read_csv('../input/Womens Clothing E-Commerce Reviews.csv')

In [71]:
df = df[['Review Text','Rating']]
df.head()

,Review Text,Rating
0,Absolutely wonderful - silky and sexy and comf...,4
1,Love this dress! it's sooo pretty. i happene...,5
2,I had such high hopes for this dress and reall...,3
3,"I love, love, love this jumpsuit. it's fun, fl...",5
4,This shirt is very flattering to all due to th...,5


In [72]:
df = df.dropna(how='any',axis=0)

In [73]:
X = df.iloc[:,0].values
y = df.iloc[:,1].values

In [74]:
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

In [75]:
stop = stopwords.words('english')
def ponctuation(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text)
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))    
    return text

In [76]:
for i in range(0,len(X)):
    X[i] = ponctuation(X[i])

In [77]:
X[1]

'love this dress it s sooo pretty i happened to find it in a store and i m glad i did bc i never would have ordered it online bc it s petite i bought a petite and am 5 8 i love the length on me hits just a little below the knee would definitely be a true midi on someone who is truly petite '

In [78]:
stemmer = PorterStemmer()
def stemmering(text):
    return [stemmer.stem(w) for w in text.split()]

In [79]:
def stem(text):
    text_stem = [w for w in stemmering(text) if w not in stop]
    new_text = ''
    for i in text_stem[0:]:
        new_text = new_text + i + ' '
    return new_text

In [80]:
for i in range(0,len(X)):
    X[i] = stem(X[i])

In [81]:
X[1]

'love thi dress sooo pretti happen find store glad bc never would order onlin bc petit bought petit 5 8 love length hit littl knee would definit true midi someon truli petit '

In [82]:
count_vect = CountVectorizer()
X_counts = count_vect.fit_transform(X)

In [83]:
tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X_counts)
X_tfidf.shape

(22641, 9781)

## Questao 2 b

In [84]:
X_t = X_tfidf.toarray()

In [85]:
X_total, X_test, y_total, y_test = train_test_split(X_t,y,test_size=0.1,random_state=42)

In [86]:
X_train, X_val, y_train, y_val = train_test_split(X_total,y_total,test_size=0.1,random_state=42)

In [20]:
model = MLPClassifier()

In [27]:
model.fit(X_train, y_train).score(X_val, y_val)

0.5686947988223748

In [28]:
y_pred = model.score(X_test,y_test)
y_pred

0.5766004415011038

# questao 2 c

In [100]:
model_f = Sequential()
model_f.add(Conv1D(64, 5, input_shape=(9781, 1)))
model_f.add(Conv1D(32, 5))
model_f.add(Dense(5, activation='softmax'))
model_f.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_29 (Conv1D)           (None, 9777, 64)          384       
_________________________________________________________________
conv1d_30 (Conv1D)           (None, 9773, 32)          10272     
_________________________________________________________________
dense_29 (Dense)             (None, 9773, 5)           165       
Total params: 10,821
Trainable params: 10,821
Non-trainable params: 0
_________________________________________________________________


In [87]:
X_train = np.expand_dims(X_train, axis=2) 
X_val = np.expand_dims(X_val, axis=2) 
X_test = np.expand_dims(X_test, axis=2)

In [98]:
X_train.shape, X_val.shape, X_test.shape

((18338, 9781, 1), (2038, 9781, 1), (2265, 9781, 1))

In [101]:
model_f.compile(optimizer='rmsprop',loss='sparse_categorical_crossentropy', metrics=['acc'])
history = model_f.fit(X_train, y_train, 
                    epochs=20, 
                    batch_size=128,
                    validation_data=(X_val,y_val))

ValueError: Error when checking target: expected dense_29 to have 3 dimensions, but got array with shape (18338, 1)